# Load the Twitter Dataset

In [4]:
import csv
tweets_list = []

with open('../Datasets/new_twitter_data_men_rem_to_send.csv') as file:
    dataset_csv_reader =  csv.reader(file)
    
    # Get headers
    header = []
    header = next(dataset_csv_reader)

    #Extract tweets into list
    tweets_list = []
    for row in dataset_csv_reader:
        tweets_list.append(row[1])
    
    #Print first 3 entries
    for tweet in tweets_list[:3]:
        print(f'---- {tweet}')
    

----  Can I know wen r you gonna reply.  I called up ur cust care sum days bk n dey asked me 2 send an email 2 contact.in since it's related 2 offers.  I send dem an email n 2day dey ask me 2 contact sales team since dis was at da time of sales.FIFA fever is on in ur office!
---- RT : .     
I can't understand,Why so many issues with Dell La…
---- A Win for America: Dell to Phase Out All Computer Chips Produced in China

It’s about time. I hope this marks a trend. 
Thank you, ⁦⁩.  https://t.co/CZZwmQF9Bf


## Run the text through Spacy Accuracy Model and Convert To Label Studio Tasks

In [5]:
import spacy
from itertools import groupby
import json

nlp = spacy.load("en_core_web_trf")

# This function converts spaCy docs to the list of named entity spans in Label Studio compatible JSON format:
def doc_to_spans(doc):
    tokens = [(tok.text, tok.idx, tok.ent_type_) for tok in doc]
    results = []
    entities = set()
    for entity, group in groupby(tokens, key=lambda t: t[-1]):
        if not entity:
            continue
        group = list(group)
        _, start, _ = group[0]
        word, last, _ = group[-1]
        text = ' '.join(item[0] for item in group)
        end = last + len(word)
        results.append({
            'from_name': 'label',
            'to_name': 'text',
            'type': 'labels',
            'value': {
                'start': start,
                'end': end,
                'text': text,
                'labels': [entity]
            }
        })
        entities.add(entity)

    return results, entities

# Prepare Label Studio tasks in import JSON format with the model predictions:
entities = set()
tasks = []
for text in tweets_list:
    predictions = []
    doc = nlp(text)
    spans, ents = doc_to_spans(doc)
    entities |= ents
    predictions.append({'model_version': 'en_core_web_lg', 'result': spans})
    tasks.append({
        'data': {'text': text},
        'predictions': predictions
    })

# Save Label Studio tasks.json
print(f'Save {len(tasks)} tasks to "tasks.json"')
with open('twitter_tasks.json', mode='w') as f:
    json.dump(tasks, f, indent=2)
    
# Save class labels as a txt file
print('Named entities are saved to "named_entities.txt"')
with open('named_entities_twitter.txt', mode='w') as f:
    f.write('\n'.join(sorted(entities)))

    

Save 2857 tasks to "tasks.json"
Named entities are saved to "named_entities.txt"


## Load the Reddit Dataset

In [4]:
import csv

with open('../../Datasets/dell_output_tcd_processed.csv') as file:
    dataset_csv_reader =  csv.reader(file)
    
    # Get headers
    header = []
    header = next(dataset_csv_reader)

    #Extract tweets into list
    text_list = []
    for row in dataset_csv_reader:
        text_list.append(row[0])
    
    #Print first 3 entries
    for text in text_list[:3]:
        print(f'---- {text}')

---- Lock Kensigton for XPS
---- Hello! What do you think about it? I would like to buy this lock for when I use my laptop on the go. Does it have any contraindications? Do you think it can easily be folded for carrying in a backpack?

Or it would be better for transport to have a lock with a spiral wire (but I can't find compatibility with XPS 9720) ...

[https://www.dell.com/en-us/shop/slim-n17-20-keyed-dual-head-laptop-lock-for-wedge-shaped-slots/apd/ab868415/pc-accessories#techspecs\_section](https://www.dell.com/en-us/shop/slim-n17-20-keyed-dual-head-laptop-lock-for-wedge-shaped-slots/apd/ab868415/pc-accessories#techspecs_section)

Thank you very much.

Edit:  \*Kensington

Edit: do you know if these products are also shipped from China, or is there a stock in Europe?

&#x200B;

https://preview.redd.it/e48fjc3yatx91.png?width=3317&format=png&auto=webp&v=enabled&s=6c2253497e182c3c6f9b299ba06b382a5ea50ae6

&#x200B;
---- Dell Latitude 5490 suggestions for LCD upgrade


## Run the text through Spacy Accuracy Model and Convert To Label Studio Tasks

In [5]:
import spacy
from itertools import groupby
import json

nlp = spacy.load("en_core_web_trf")

# This function converts spaCy docs to the list of named entity spans in Label Studio compatible JSON format:
def doc_to_spans(doc):
    tokens = [(tok.text, tok.idx, tok.ent_type_) for tok in doc]
    results = []
    entities = set()
    for entity, group in groupby(tokens, key=lambda t: t[-1]):
        if not entity:
            continue
        group = list(group)
        _, start, _ = group[0]
        word, last, _ = group[-1]
        text = ' '.join(item[0] for item in group)
        end = last + len(word)
        results.append({
            'from_name': 'label',
            'to_name': 'text',
            'type': 'labels',
            'value': {
                'start': start,
                'end': end,
                'text': text,
                'labels': [entity]
            }
        })
        entities.add(entity)

    return results, entities

# Prepare Label Studio tasks in import JSON format with the model predictions:
entities = set()
tasks = []
for text in text_list:
    predictions = []
    doc = nlp(text)
    spans, ents = doc_to_spans(doc)
    entities |= ents
    predictions.append({'model_version': 'en_core_web_trf', 'result': spans})
    tasks.append({
        'data': {'text': text},
        'predictions': predictions
    })

# Save Label Studio tasks.json
print(f'Save {len(tasks)} tasks to "reddit_tasks.json"')
with open('reddit_tasks.json', mode='w') as f:
    json.dump(tasks, f, indent=2)
    
# Save class labels as a txt file
print('Named entities are saved to "named_entities_reddit.txt"')
with open('named_entities_reddit.txt', mode='w') as f:
    f.write('\n'.join(sorted(entities)))

Token indices sequence length is longer than the specified maximum sequence length for this model (537 > 512). Running this sequence through the model will result in indexing errors


Save 4179 tasks to "reddit_tasks.json"
Named entities are saved to "named_entities_reddit.txt"
